In [ ]:
%matplotlib inline

from dask.distributed import Client
import dask.dataframe as dd
import matplotlib.pyplot as plt
import joblib
import numpy as np

In [ ]:
# Start and connect to local client

client = Client(n_workers=4)
# client = Client("scheduler-address:8786")  # connecting to remote cluster

In [ ]:
client

In [ ]:
def read_file(filepath):
    df = dd.read_csv(filepath, sep = ',', header = None)
    df.columns = ['subject_id', 'activity_code', 'timestamp', 'x_coord', 'y_coord', 'z_coord']
    df['z_coord'] = df['z_coord'].str.replace(";","").astype('float64') # remove ; and ensure float (having issues with lineterminator)
    return df

In [ ]:
phone_gyro_df = read_file('wisdm-dataset/raw/phone/gyro/*.txt')
phone_accel_df = read_file('wisdm-dataset/raw/phone/accel/*.txt')
watch_gyro_df = read_file('wisdm-dataset/raw/watch/gyro/*.txt')
watch_accel_df = read_file('wisdm-dataset/raw/watch/accel/*.txt')

In [ ]:
len(phone_gyro_df[ (phone_gyro_df['subject_id'] == 1600) & (phone_gyro_df['activity_code'] == 'A') ])

In [ ]:
client.shutdown()